* ⊕ [将 Pandas DataFrame 转换为 JSON | D栈 - Delft Stack](https://www.delftstack.com/zh/howto/python-pandas/pandas-dataframe-to-json/)

In [3]:
from clickhouse_driver import connect
from clickhouse_driver import Client

client = Client('localhost', settings={'use_numpy': True})
sql='''SELECT toYear(date) AS year, round(avg(price)) AS price, bar(price, 0, 1000000, 80) FROM uk_price_paid GROUP BY year ORDER BY year;
'''
client.query_dataframe(sql)

,year,price,bar_round_avg_price____0__1000000__80_
0,2019,351037.0,████████████████████████████
1,2020,369561.0,█████████████████████████████▌
2,2021,382218.0,██████████████████████████████▌


In [4]:
# Average Price per Year in London
sql='''SELECT toYear(date) AS year, round(avg(price)) AS price, 
    bar(price, 0, 2000000, 100) FROM uk_price_paid 
    WHERE town = 'LONDON' GROUP BY year ORDER BY year;
'''
client.query_dataframe(sql)

,year,price,bar_round_avg_price____0__2000000__100_
0,2019,1042497.0,██████████████████████████████████████████████...
1,2020,1011010.0,██████████████████████████████████████████████...
2,2021,938039.0,██████████████████████████████████████████████▊


In [5]:
def query(sql):
    return client.query_dataframe(sql)

In [7]:
# The Most Expensive Neighborhoods
df=query('''SELECT
    town,
    district,
    count() AS c,
    round(avg(price)) AS price,
    bar(price, 0, 5000000, 100)
FROM uk_price_paid
WHERE date >= '2020-01-01'
GROUP BY
    town,
    district
HAVING c >= 100
ORDER BY price DESC
LIMIT 100;
''')
df

,town,district,c,price,bar_round_avg_price____0__5000000__100_
0,LONDON,CITY OF WESTMINSTER,3821,3187714.0,██████████████████████████████████████████████...
1,LONDON,CITY OF LONDON,281,3113362.0,██████████████████████████████████████████████...
2,LONDON,KENSINGTON AND CHELSEA,2683,2317014.0,██████████████████████████████████████████████▎
3,LEATHERHEAD,ELMBRIDGE,122,1879995.0,█████████████████████████████████████▌
4,VIRGINIA WATER,RUNNYMEDE,168,1766634.0,███████████████████████████████████▎
...,...,...,...,...,...
95,NORTHWOOD,HILLINGDON,295,690058.0,█████████████▋
96,CHISLEHURST,BROMLEY,462,687633.0,█████████████▋
97,WADHURST,WEALDEN,135,680979.0,█████████████▌
98,WOKING,SURREY HEATH,299,680785.0,█████████████▌


In [13]:
df=query('''SELECT
    town,
    district,
    count() AS c,
    round(avg(price)) AS price
FROM uk_price_paid
WHERE date >= '2020-01-01'
GROUP BY
    town,
    district
HAVING c >= 100
ORDER BY price DESC
LIMIT 100;
''')
df

,town,district,c,price
0,LONDON,CITY OF WESTMINSTER,3821,3187714.0
1,LONDON,CITY OF LONDON,281,3113362.0
2,LONDON,KENSINGTON AND CHELSEA,2683,2317014.0
3,LEATHERHEAD,ELMBRIDGE,122,1879995.0
4,VIRGINIA WATER,RUNNYMEDE,168,1766634.0
...,...,...,...,...
95,NORTHWOOD,HILLINGDON,295,690058.0
96,CHISLEHURST,BROMLEY,462,687633.0
97,WADHURST,WEALDEN,135,680979.0
98,WOKING,SURREY HEATH,299,680785.0


In [14]:
df[:1]

,town,district,c,price
0,LONDON,CITY OF WESTMINSTER,3821,3187714.0


In [21]:
import pandas as pd
import json
ds=df[:3]
js = ds.to_json(orient = 'records')
json.loads(js)

[{'town': 'LONDON',
  'district': 'CITY OF WESTMINSTER',
  'c': 3821,
  'price': 3187714.0},
 {'town': 'LONDON',
  'district': 'CITY OF LONDON',
  'c': 281,
  'price': 3113362.0},
 {'town': 'LONDON',
  'district': 'KENSINGTON AND CHELSEA',
  'c': 2683,
  'price': 2317014.0}]

In [25]:
from feeder import Publisher

In [26]:
publisher = Publisher()
publisher.publish_multi('price_paid', json.loads(js))

```bash
$ just create price_paid
$ just list price_paid
```

In [28]:
query("select * from uk_price_paid limit 10")

,price,date,postcode1,postcode2,type,is_new,duration,addr1,addr2,street,locality,town,district,county,category
0,330000,2021-03-03,,,semi-detached,0,freehold,1,,HALL LANE,BRANDON,GRANTHAM,SOUTH KESTEVEN,LINCOLNSHIRE,0
1,250000,2021-05-19,,,detached,0,freehold,1,,SPRING HILL,DINAS,NEWPORT,PEMBROKESHIRE,PEMBROKESHIRE,0
2,197000,2021-03-22,,,detached,0,freehold,1,,ASHCROFT MANOR CLOSE,ALMELEY,HEREFORD,HEREFORDSHIRE,HEREFORDSHIRE,0
3,165000,2021-07-23,,,terraced,0,freehold,1,,THE GREEN,HICKLING,MELTON MOWBRAY,RUSHCLIFFE,NOTTINGHAMSHIRE,0
4,365000,2021-02-26,,,semi-detached,0,freehold,1,,HIGH STREET,BREAM,LYDNEY,FOREST OF DEAN,GLOUCESTERSHIRE,1
5,635000,2021-08-19,,,detached,0,freehold,1 OLD SCHOOL PLACE,,COCKLETON LANE,GURNARD,COWES,ISLE OF WIGHT,ISLE OF WIGHT,0
6,170000,2021-03-09,,,detached,0,freehold,10,,SOUTH ROAD,,WOOLER,NORTHUMBERLAND,NORTHUMBERLAND,1
7,1925000,2021-02-25,,,other,0,freehold,10,,CRAVEN PARK ROAD,,LONDON,HARINGEY,GREATER LONDON,1
8,110000,2021-01-08,,,flat,0,leasehold,10,10,DENE HOUSE COURT,,LEEDS,LEEDS,WEST YORKSHIRE,1
9,59950,2021-04-07,,,flat,0,leasehold,10,FLAT,WESTBOURNE GROVE,SKETTY,SWANSEA,SWANSEA,SWANSEA,0


In [29]:
ds=query("select * from uk_price_paid limit 10")
js = ds.to_json(orient = 'records')
publisher.publish_multi('price_paid', json.loads(js))